**Table of contents**<a id='toc0_'></a>    
- [Importation des bibliothèques](#toc1_)    
  - [Importation des paquets ou modules de la bibliothèque OpenFL](#toc1_1_)    
  - [Importation des paquets ou modules de la bibliothèque PyTorch](#toc1_2_)    
  - [Importation d’autres paquets ou modules requis](#toc1_3_)    
- [Entraînement de modèle de réseau CNN](#toc2_)    
  - [Définition des chargeurs de données](#toc2_1_)    
  - [Définition du modèle de réseau CNN](#toc2_2_)    
  - [Définition de la formule d'inférence utilisée dans le test](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Importation des bibliothèques](#toc0_)

## <a id='toc1_1_'></a>[Importation des paquets ou modules de la bibliothèque OpenFL](#toc0_)

In [1]:
from openfl.experimental.workflow.interface import Aggregator, Collaborator, FLSpec
from openfl.experimental.workflow.placement import aggregator, collaborator
from openfl.experimental.workflow.runtime import LocalRuntime

## <a id='toc1_2_'></a>[Importation des paquets ou modules de la bibliothèque PyTorch](#toc0_)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary

import torchvision

## <a id='toc1_3_'></a>[Importation d’autres paquets ou modules requis](#toc0_)

In [3]:
from copy import deepcopy

import numpy as np

# <a id='toc2_'></a>[Entraînement de modèle de réseau CNN](#toc0_)

## <a id='toc2_1_'></a>[Définition des chargeurs de données](#toc0_)

In [4]:
"""
01. torchvision.transforms.Compose(transforms)
    - Composes several transforms together.

02. torchvision.transforms.Normalize(mean, std, inplace=False)
    - Normalize a tensor image with mean and standard deviation.
    - output[channel] = (input[channel] - mean[channel]) / std[channel]
"""

mnist_train = torchvision.datasets.MNIST(
    "/tmp/files/",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            # Les valeurs ` 0.1307` et `0.3081` utilisées pour la transformation `Normalize()`
            # ci-dessous sont la moyenne globale et l’écart-type de l’ensemble de données MNIST.
            torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        ]
    ),
)

mnist_test = torchvision.datasets.MNIST(
    "/tmp/files/",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        ]
    ),
)

## <a id='toc2_2_'></a>[Définition du modèle de réseau CNN](#toc0_)

In [ ]:
"""
03. torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
    groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
    - Applies a 2D convolution over an input signal composed of several input planes.

04. torch.nn.Dropout2d(p=0.5, inplace=False)
    - Randomly zero out entire channels.
    - Each channel will be zeroed out independently on every forward call with probability p using
    samples from a Bernoulli distribution.

05. torch.nn.functional.max_pool2d(input, kernel_size, stride=None, padding=0,
    dilation=1, ceil_mode=False, return_indices=False)
    - Applies a 2D max pooling over an input signal composed of several input planes.

06. torch.nn.functional.dropout(input, p=0.5, training=True, inplace=False)
    - During training, randomly zeroes some elements of the input tensor with probability p.
    - Uses samples from a Bernoulli distribution.

07. torch.nn.functional.log_softmax(input, dim=None, _stacklevel=3, dtype=None)
    - Apply a softmax followed by a logarithm.
    - While mathematically equivalent to log(softmax(x)), doing these two operations separately is
    slower and numerically unstable. This function uses an alternative formulation to compute the
    output and gradient correctly.
"""


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # La première couche convolutionnelle : le nombre de canaux d’entrée est de 1, c’est-à-dire
        # une image en niveaux de gris, le nombre de canaux de sortie est de 10, la taille du filtre
        # convolutif est de 5x5, le stride est de 1 et le padding est de 0.

        # Par conséquent, après que l'image d'entrée (1x28x28) a été convoluée, la taille de la
        # carte de caractéristiques de sortie est de 10x24x24.
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # La deuxième couche convolutionnelle : le nombre de canaux d’entrée est de 10, le nombre de
        # canaux de sortie est de 20, la taille du filtre convolutif est de 5x5, le stride est de 1
        # et le padding est de 0.

        # Par conséquent, après que l'image d'entrée (10x12x12) a été convoluée, la taille de la
        # carte de caractéristiques de sortie est de 20x8x8.
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # Une couche d'abandon.
        self.conv2_drop = nn.Dropout2d()
        # La première couche de fully connected : le nombre de canaux d’entrée est de 20, chaque
        # canal a une taille de 4x4, soit un total de 20x4x4 = 320 nœuds, tandis que la sortie est
        # fixée à 50 nœuds.
        self.fc1 = nn.Linear(320, 50)
        # La deuxième couche de fully connected : l'entrée a 50 nœuds et la sortie a 10 nœuds
        # (correspondant à 10 catégories).
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # La première couche convolutive est suivie d'une couche de pooling de type max pooling avec
        # un filtre convolutif de taille de 2x2 et un stride égal à la longueur du filtre.

        # La taille d'entrée est de 10x24x24, et après pooling, la taille de sortie est de 10x12x12.
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # La deuxième couche convolutive est suivie d'une couche d'abandon.

        # Après la couche d'abandon, suit une autre couche de max-pooling, qui possède un filtre
        # convolutif de taille de 2x2 et un stride aussi égal à la longueur du filtre.

        # La taille d'entrée est de 20x8x8, et après pooling, la taille de sortie est de 20x4x4.
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # La carte de caractéristiques multidimensionnelle est transformée en un vecteur
        # unidimensionnel, d'une taille de 20x4x4 = 320.
        x = x.view(-1, 320)
        # La première couche de fully connected, activée par la fonction d'activation ReLU.
        x = F.relu(self.fc1(x))
        # Pendant l'entraînement du modèle, certains nœuds de la sortie de la première couche de
        # fully connected sont mis à zéro de manière aléatoire avec une probabilité p.
        x = F.dropout(x, training=self.training)
        # La deuxième couche de fully connected sert également de couche de sortie.
        x = self.fc2(x)
        # Les probabilités logarithmiques de tous les nœuds de la couche de sortie sont calculées en
        # appliquant une fonction softmax suivie d'un logarithme.
        return F.log_softmax(x, dim=1)

In [6]:
model = Net()
summary(model, next(iter(mnist_train))[0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------


## <a id='toc2_3_'></a>[Définition de la formule d'inférence utilisée dans le test](#toc0_)

In [ ]:
"""
08. torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100,
    reduce=None, reduction='mean')
    - Compute the negative log likelihood loss.
"""


def inference(network, test_loader):
    # Mettre le module en mode évaluation.
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            # Le log-vraisemblance négatif est également connu sous le nom d'entropie croisée
            # catégorielle, car il s'agit en fait de deux interprétations différentes de la même
            # formule.

            # L’entropie croisée catégorielle sert au classement en plusieurs classes.

            # L'entropie est une mesure de l'incertitude, c'est-à-dire que, si un résultat est
            # certain, l'entropie est faible.

            # La perte d'entropie croisée, ou perte logarithmique mesure les performances d'un
            # modèle de classification dont le résultat est une valeur de probabilité comprise entre
            # 0 et 1.

            # La perte d'entropie croisée augmente à mesure que la probabilité prédite s'écarte de
            # l'étiquette réelle.
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            # Si `keepdim` est `True`, le tenseur de sortie est de la même taille que celui
            # d'entrée, sauf dans la (les) dimension(s) `dim` où il est de taille 1.
            pred = output.data.max(dim=1, keepdim=True)[1]
            # Calcul de l'égalité par éléments.
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    print(
        "\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    accuracy = float(correct / len(test_loader.dataset))
    return accuracy

In [12]:
inference(model, mnist_test)

/usr/local/lib/python3.11/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


TypeError: nll_loss_nd(): argument 'target' (position 2) must be Tensor, not int

In [13]:
for data, target in mnist_test:
    print(target)

7
2
1
0
4
1
4
9
5
9
0
6
9
0
1
5
9
7
3
4
9
6
6
5
4
0
7
4
0
1
3
1
3
4
7
2
7
1
2
1
1
7
4
2
3
5
1
2
4
4
6
3
5
5
6
0
4
1
9
5
7
8
9
3
7
4
6
4
3
0
7
0
2
9
1
7
3
2
9
7
7
6
2
7
8
4
7
3
6
1
3
6
9
3
1
4
1
7
6
9
6
0
5
4
9
9
2
1
9
4
8
7
3
9
7
4
4
4
9
2
5
4
7
6
7
9
0
5
8
5
6
6
5
7
8
1
0
1
6
4
6
7
3
1
7
1
8
2
0
2
9
9
5
5
1
5
6
0
3
4
4
6
5
4
6
5
4
5
1
4
4
7
2
3
2
7
1
8
1
8
1
8
5
0
8
9
2
5
0
1
1
1
0
9
0
3
1
6
4
2
3
6
1
1
1
3
9
5
2
9
4
5
9
3
9
0
3
6
5
5
7
2
2
7
1
2
8
4
1
7
3
3
8
8
7
9
2
2
4
1
5
9
8
7
2
3
0
4
4
2
4
1
9
5
7
7
2
8
2
6
8
5
7
7
9
1
8
1
8
0
3
0
1
9
9
4
1
8
2
1
2
9
7
5
9
2
6
4
1
5
8
2
9
2
0
4
0
0
2
8
4
7
1
2
4
0
2
7
4
3
3
0
0
3
1
9
6
5
2
5
9
2
9
3
0
4
2
0
7
1
1
2
1
5
3
3
9
7
8
6
5
6
1
3
8
1
0
5
1
3
1
5
5
6
1
8
5
1
7
9
4
6
2
2
5
0
6
5
6
3
7
2
0
8
8
5
4
1
1
4
0
3
3
7
6
1
6
2
1
9
2
8
6
1
9
5
2
5
4
4
2
8
3
8
2
4
5
0
3
1
7
7
5
7
9
7
1
9
2
1
4
2
9
2
0
4
9
1
4
8
1
8
4
5
9
8
8
3
7
6
0
0
3
0
2
6
6
4
9
3
3
3
2
3
9
1
2
6
8
0
5
6
6
6
3
8
8
2
7
5
8
9
6
1
8
4
1
2
5
9
1
9
7
5
4
0
8
9
9
1
0
5
2
3
7
8
9
4
0
6
